# Recurrent Neural Networks RNN

Recurrent neural networks, also known as RNNs, are a class of neural networks that allow previous outputs to be used as inputs while having hidden states.


Applications: Mostly used in language processing and speach recognition

https://classroom.udacity.com/courses/ud188/lessons/a8fc0724-37ed-40d9-a226-57175b8bb8cc/concepts/284fab25-93cb-4192-840e-371f0c75cc07

Now that you have some experience with PyTorch and deep learning, I'll be teaching you about recurrent neural networks (RNNs) and long short-term memory (LSTM) . RNNs are designed specifically to learn from sequences of data by passing the hidden state from one step in the sequence to the next step in the sequence, combined with the input. LSTMs are an improvement the RNNs, and are quite useful when our neural network needs to switch between remembering recent things, and things from long time ago. But first, I want to give you some great references to study this further. There are many posts out there about LSTMs, here are a few of my favorites:

http://colah.github.io/posts/2015-08-Understanding-LSTMs/
http://blog.echen.me/2017/05/30/exploring-lstms/

http://karpathy.github.io/2015/05/21/rnn-effectiveness/
https://www.youtube.com/watch?v=iX5V1WpxxkY


# Character-Level LSTM in PyTorch
LSTM

We cannot close any post that tries to look at what RNNs and related architectures are without mentioning LSTMs. This is not a different variant of RNN architecture, but rather it introduces changes to how we compute outputs and hidden state using the inputs.
In a vanilla RNN, the input and the hidden state are simply passed through a single tanh layer. LSTM (Long Short Term Memory) networks improve on this simple transformation and introduces additional gates and a cell state, such that it fundamentally addresses the problem of keeping or resetting context, across sentences and regardless of the distance between such context resets. There are variants of LSTMs including GRUs that utilize the gates in different manners to address the problem of long term dependencies.


https://classroom.udacity.com/courses/ud188/lessons/a8fc0724-37ed-40d9-a226-57175b8bb8cc/concepts/e0568d87-96f4-4270-8552-22dc05a1e4f4


http://localhost:8888/tree/Documents/olga_data_science_udacity_deep_learning_pytorch/recurrent-neural-networks/char-rnn



In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [5]:
encoded[:100]

array([70, 23, 72, 17, 34, 10, 50,  1, 65,  4,  4,  4, 58, 72, 17, 17, 46,
        1, 62, 72, 42, 75, 39, 75, 10, 41,  1, 72, 50, 10,  1, 72, 39, 39,
        1, 72, 39, 75, 78, 10, 38,  1, 10, 12, 10, 50, 46,  1, 57, 81, 23,
       72, 17, 17, 46,  1, 62, 72, 42, 75, 39, 46,  1, 75, 41,  1, 57, 81,
       23, 72, 17, 17, 46,  1, 75, 81,  1, 75, 34, 41,  1, 32, 66, 81,  4,
       66, 72, 46, 27,  4,  4,  6, 12, 10, 50, 46, 34, 23, 75, 81])

In [6]:
print(int2char)

{0: 'V', 1: ' ', 2: 'z', 3: 'F', 4: '\n', 5: 'T', 6: 'E', 7: '%', 8: '0', 9: '2', 10: 'e', 11: '8', 12: 'v', 13: 'O', 14: '-', 15: 'g', 16: '@', 17: 'p', 18: ')', 19: 'I', 20: '$', 21: 'q', 22: ':', 23: 'h', 24: 'j', 25: 'd', 26: ',', 27: '.', 28: 'Z', 29: 'J', 30: '6', 31: 'R', 32: 'o', 33: 'B', 34: 't', 35: 'P', 36: 'N', 37: 'D', 38: ';', 39: 'l', 40: 'Y', 41: 's', 42: 'm', 43: 'W', 44: 'S', 45: '7', 46: 'y', 47: 'c', 48: '!', 49: '9', 50: 'r', 51: 'M', 52: '4', 53: "'", 54: '`', 55: 'A', 56: 'Q', 57: 'u', 58: 'H', 59: '(', 60: '?', 61: '*', 62: 'f', 63: '5', 64: 'G', 65: '1', 66: 'w', 67: 'b', 68: '&', 69: 'L', 70: 'C', 71: '3', 72: 'a', 73: 'U', 74: 'X', 75: 'i', 76: '_', 77: 'K', 78: 'k', 79: '/', 80: '"', 81: 'n', 82: 'x'}


In [7]:
print(char2int)

{'V': 0, ' ': 1, 'z': 2, 'F': 3, '\n': 4, 'T': 5, 'E': 6, '%': 7, '0': 8, '2': 9, 'e': 10, '8': 11, 'v': 12, 'O': 13, '-': 14, 'g': 15, '@': 16, 'p': 17, ')': 18, 'I': 19, '$': 20, 'q': 21, ':': 22, 'h': 23, 'j': 24, 'd': 25, ',': 26, '.': 27, 'Z': 28, 'J': 29, '6': 30, 'R': 31, 'o': 32, 'B': 33, 't': 34, 'P': 35, 'N': 36, 'D': 37, ';': 38, 'l': 39, 'Y': 40, 's': 41, 'm': 42, 'W': 43, 'S': 44, '7': 45, 'y': 46, 'c': 47, '!': 48, '9': 49, 'r': 50, 'M': 51, '4': 52, "'": 53, '`': 54, 'A': 55, 'Q': 56, 'u': 57, 'H': 58, '(': 59, '?': 60, '*': 61, 'f': 62, '5': 63, 'G': 64, '1': 65, 'w': 66, 'b': 67, '&': 68, 'L': 69, 'C': 70, '3': 71, 'a': 72, 'U': 73, 'X': 74, 'i': 75, '_': 76, 'K': 77, 'k': 78, '/': 79, '"': 80, 'n': 81, 'x': 82}


## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [8]:
#function take an encoded array(arr) and turns it into one-hot vector of some specified length
# n_labels - specified length of vector
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [9]:
#check how function above works with array of 3 numbers
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


Result is an array of 3 one-hot vectors. All these vectors have length 8 and index 3
For our vocabulary of 83 vectors our array will be just much longer vectors


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="data/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.



In [10]:
# arr is array of some encoded data
# batch_size and seq_length
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    #first calculate how many characters were in a complete mini-batch
    batch_size_total = batch_size * seq_length
    ## TODO: Get the number of batches we can make
    #length of the array divided by the total number of characters in a mini_batch
    # // is integer division for rounding result of the division
    n_batches = len(arr)//batch_size_total
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    
    ## TODO: Reshape into batch_size rows
    #number of rows will equal batch_size
    #-1 is dimension place holder, it will just automatically fill up the second dimension 
    arr = arr.reshape((batch_size, -1))
    
    ## TODO: Iterate over the batches using a window of length seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        #reshaped complete array, looking at rows, and columns are in a range from n to n_seq_length
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        #itinialize zeroes array that is same shape as x
        #fill up with values from x shifted by 1
        #last element of y is the first element of x
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]        
        
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [11]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [12]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[44 56 14 75 47 25 52 31 13 36]
 [26 66 40 31 47 56 14 47 31 14]
 [25 40  1 31 66 52 31 14 31  4]
 [26 31 47 56 25 31 49 56 39 25]
 [31 26 14 58 31 56 25 52 31 47]
 [49 59 26 26 39 66 40 31 14 40]
 [31  2 40 40 14 31 56 14  1 31]
 [45 38 61 66 40 26 48 50 42 31]]

y
 [[56 14 75 47 25 52 31 13 36 36]
 [66 40 31 47 56 14 47 31 14 47]
 [40  1 31 66 52 31 14 31  4 66]
 [31 47 56 25 31 49 56 39 25  4]
 [26 14 58 31 56 25 52 31 47 25]
 [59 26 26 39 66 40 31 14 40  1]
 [ 2 40 40 14 31 56 14  1 31 26]
 [38 61 66 40 26 48 50 42 31  7]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="data/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [23]:
# check if GPU is available
#train_on_gpu = torch.cuda.is_available()
#if(train_on_gpu):
#    print('Training on GPU!')
#else: 
#    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [28]:
#code for defininf character level on RNN
class CharRNN(nn.Module):
    
    #specify n_hidden - hidden dimension, number of layers, and dropout probability
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM layer
        # this takes input size (len(self.chars), length of one hot encoded input character)
        #takes numger of hidden dimenstion (n_hidden), number of layers and drop out probability
        #this will create a dropout layer in between multiple LSTM layers
        #the first dimension is the batch_size
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        #to go in-between my LSTM and final linear layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        #this takes in LSTM outputs which will be n_hidden
        #will output class scores for the most likely next character
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        #-1 means I'am going to be stacking up the outputs of the LSTM
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        #passing V-shaped output to the final fully connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
#        if (train_on_gpu):
#            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
#                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
#        else:
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                 weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.



https://classroom.udacity.com/courses/ud188/lessons/a8fc0724-37ed-40d9-a226-57175b8bb8cc/concepts/284fab25-93cb-4192-840e-371f0c75cc07


In [30]:
#this function takes in model to train, some data and number of epochs to train for

def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    #define optimizer
    #standard Adam optimizer with a learning rate set the past and learning rate(lr=0.001) 
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    #define loss function - cross entropy loss usefull for character class scores
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    #move model to GPU if it is available
#    if(train_on_gpu):
#        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    
    #here we start an epic loop
    for e in range(epochs):
        
        # initialize hidden state of our LSTM
        #takes batch_size to define a size of hidden state
        #and returns a hidden in cell state that are all zeroes
        h = net.init_hidden(batch_size)
        
        #my batch loop inside epic loop
        #this is getting our X and Y mini batches from our get batches generator
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            #converting the input into one-hot encodede represention
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            #if GPU is available moving inputs and targets to GPU device
#            if(train_on_gpu):
#                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            #making sure that we detach any past in hidden state from its history
            #hidden state of LSTM layer is a Tuple
            h = tuple([each.data for each in h])

            #back propagation as usual
            # zero out any accumulated gradients
            net.zero_grad()
            
            #pass in our input Tensors to our model and hidden state
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss (by looking at the predicted output and targets)
            #and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            #clipping gradients (set clip threshold)
            #It is a technique used to cope with the exploding gradient problem sometimes encountered 
            #when performing backpropagation. By capping the maximum value for the gradient, this phenomenon 
            #is controlled in practice.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            #moving one step in right direction updating weights of our network
            #perform optimization step
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
#                    if(train_on_gpu):
#                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [31]:
## TODO: set your model hyperparameters
# define and print the net
n_hidden=512
n_layers=2

#instantiate model and print
net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


so we have 83 unique characters as input
512 as a hidden dimension
2 layers
for a dropout layer we have a default value of 0.5 (probility)
for last our fully connected layers we have input features same as hidden dimension = 512 and output features (number of characters 84)


### Set your training hyperparameters!

In [32]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start small if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2866... Val Loss: 3.2265
Epoch: 1/20... Step: 20... Loss: 3.1528... Val Loss: 3.1399
Epoch: 1/20... Step: 30... Loss: 3.1480... Val Loss: 3.1260
Epoch: 1/20... Step: 40... Loss: 3.1189... Val Loss: 3.1192
Epoch: 1/20... Step: 50... Loss: 3.1463... Val Loss: 3.1178
Epoch: 1/20... Step: 60... Loss: 3.1185... Val Loss: 3.1150
Epoch: 1/20... Step: 70... Loss: 3.1060... Val Loss: 3.1126
Epoch: 1/20... Step: 80... Loss: 3.1192... Val Loss: 3.1055
Epoch: 1/20... Step: 90... Loss: 3.1047... Val Loss: 3.0872
Epoch: 1/20... Step: 100... Loss: 3.0643... Val Loss: 3.0522
Epoch: 1/20... Step: 110... Loss: 2.9987... Val Loss: 2.9697
Epoch: 1/20... Step: 120... Loss: 2.8883... Val Loss: 2.8920
Epoch: 1/20... Step: 130... Loss: 2.8227... Val Loss: 2.7810
Epoch: 2/20... Step: 140... Loss: 2.7318... Val Loss: 2.6910
Epoch: 2/20... Step: 150... Loss: 2.6518... Val Loss: 2.5929
Epoch: 2/20... Step: 160... Loss: 2.5580... Val Loss: 2.5205
Epoch: 2/20... Step: 170... Loss:

Epoch: 10/20... Step: 1350... Loss: 1.3781... Val Loss: 1.4156
Epoch: 10/20... Step: 1360... Loss: 1.3808... Val Loss: 1.4145
Epoch: 10/20... Step: 1370... Loss: 1.3720... Val Loss: 1.4096
Epoch: 10/20... Step: 1380... Loss: 1.4120... Val Loss: 1.4053
Epoch: 10/20... Step: 1390... Loss: 1.4149... Val Loss: 1.4063
Epoch: 11/20... Step: 1400... Loss: 1.4180... Val Loss: 1.4035
Epoch: 11/20... Step: 1410... Loss: 1.4352... Val Loss: 1.3989
Epoch: 11/20... Step: 1420... Loss: 1.4163... Val Loss: 1.3974
Epoch: 11/20... Step: 1430... Loss: 1.3881... Val Loss: 1.3983
Epoch: 11/20... Step: 1440... Loss: 1.4056... Val Loss: 1.3962
Epoch: 11/20... Step: 1450... Loss: 1.3448... Val Loss: 1.3977
Epoch: 11/20... Step: 1460... Loss: 1.3619... Val Loss: 1.3945
Epoch: 11/20... Step: 1470... Loss: 1.3511... Val Loss: 1.3948
Epoch: 11/20... Step: 1480... Loss: 1.3787... Val Loss: 1.3914
Epoch: 11/20... Step: 1490... Loss: 1.3636... Val Loss: 1.3925
Epoch: 11/20... Step: 1500... Loss: 1.3422... Val Loss:

Epoch: 20/20... Step: 2660... Loss: 1.2173... Val Loss: 1.2733
Epoch: 20/20... Step: 2670... Loss: 1.2314... Val Loss: 1.2703
Epoch: 20/20... Step: 2680... Loss: 1.2161... Val Loss: 1.2710
Epoch: 20/20... Step: 2690... Loss: 1.2103... Val Loss: 1.2734
Epoch: 20/20... Step: 2700... Loss: 1.2217... Val Loss: 1.2696
Epoch: 20/20... Step: 2710... Loss: 1.1895... Val Loss: 1.2777
Epoch: 20/20... Step: 2720... Loss: 1.1916... Val Loss: 1.2720
Epoch: 20/20... Step: 2730... Loss: 1.1717... Val Loss: 1.2685
Epoch: 20/20... Step: 2740... Loss: 1.1866... Val Loss: 1.2727
Epoch: 20/20... Step: 2750... Loss: 1.1895... Val Loss: 1.2784
Epoch: 20/20... Step: 2760... Loss: 1.1815... Val Loss: 1.2758
Epoch: 20/20... Step: 2770... Loss: 1.2231... Val Loss: 1.2778
Epoch: 20/20... Step: 2780... Loss: 1.2476... Val Loss: 1.2682


## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

The goal of this model, that it can take one character and produce a next character.

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [33]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch2.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [42]:
#function predict takes in a model=net and a character char and tries to predict next character
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        # convert input character it to integer value
        x = np.array([[net.char2int[char]]])
        #convert to one one-hot encoded representation
        x = one_hot_encode(x, len(net.chars))
        #convert this inputs into a tensor
        inputs = torch.from_numpy(x)
        
        #put inputs on gpu, if it is available
#        if(train_on_gpu):
#        inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        #returns output and new hidden state
        out, h = net(inputs, h)

        # get the character probabilities for the likely next character
        p = F.softmax(out, dim=1).data
#        if(train_on_gpu):
        p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [43]:
#takes model and size of text we want to generate, also take in prime
#which is a set of characters that we want to start our model off
def sample(net, size, prime='The', top_k=None):
        
#    if(train_on_gpu):
#        net.cuda()
#    else:
    net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [44]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna, holdon of
the strained table, and had told him the marshal and with a courtery
and stony a spilit answer, and so strung a past, and the
conversation was say to her. He said that she sent a long words, and that had asked
him to him.

The pair of his wife had said, to be at that time and a sort of thank with
his study to take his heart that was alone, and standing how there had
been a sick man, and as a face of the partical sair, she was not
stranging to her. But suncersing to a lood start and said in her looking
subject with her and his brother to see his best and way, as
he should never be daren, and that he was to draw at the conversation of his
work to his heart, and shiling to his wife. How she had no one
of the concervant would never have said to her. He flushed her hand and
with a single, though the charm and the princess and Sviazhsky had
too had been trues.

"The wine it is to be true at the carriage, but I sat down, In you
the sort of a moment improve the same--and I sat 

## Loading a checkpoint

In [45]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch2.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [46]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said
when she had astertured to him, which had not been to go into his wife
with a long while and set of all them, and the point of herself there had
been differently, what had to gate him, and he could not say
that thes was no one of his feeling that was to be children and to be
day, to be a little stears, and asked an one one of this, he stattered
in a shaking post in a low of society would have been beaned to the carped and
standing this work, he had taken all about his words on the satisfaction of
an all that had been thrunthing on home.

"Toon a presture to make it, and you come to her and make up the same
sense it where's both that woman, the princess's criminal, and I see you,
then this is in the contrary of the simplicity of the peapants, and tried to be
all all about that with, and how to be supporing, though, I
said I ceased to be told them in the children in the marsh. But, that'll
do so that I am not seening it, to decoses me," said Stepan
Arkadyevitch with his ha